In [2]:
import pandas as pd
import json

df = pd.read_csv('../data/df_problems_tags.csv', index_col=0)

In [6]:
with open('../data/dummy/accountdata.json', 'r', encoding='utf-8') as f:
    json_data = json.load(f)

In [50]:
dum_raw = json_data['solved']

In [52]:
dum_raw = pd.DataFrame(dum_raw)

In [53]:
dum_raw.head(3)

,id,status
0,1000,solved
1,2525,solved
2,7287,solved


In [57]:
# 푼 문제 id 리스트
solved_problems = list(dum_raw.loc[dum_raw['status']=='solved']['id'])

In [54]:
dum = pd.merge(dum_raw, df, how='left', left_on='id', right_on='problemId')

In [58]:
# 푼 문제 목록 추출
solved_list = dum.loc[dum['status']=='solved'].sort_values('id')
# 태그별로 문제 적게 푼 순으로 태그 나열
solved_list = solved_list.groupby('bojTagId').count().sort_values('problemId', ascending=False).reset_index()['bojTagId']
# list 형태로 변형
solved_list = list(solved_list)

In [59]:
# 푼 문제 평균 레벨 추출
solved_levels = {}

for solved_tag in solved_list:
    cond1 = (dum['status']=='solved')
    cond2 = (dum['bojTagId']==solved_tag)
    cond_list = list(dum.loc[cond1&cond2]['level'])
    solved_levels[solved_tag] = int(sum(cond_list)/len(cond_list))

In [60]:
# 시도 문제 목록 추출
tried_list = dum.loc[dum['status']=='tried'].sort_values('problemId')
# 태그별로 시도 많은 순으로 태그 나열
tried_list = tried_list.groupby('bojTagId').count().sort_values('problemId', ascending=True).reset_index()['bojTagId']
# list 형태로 변형
tried_list = list(tried_list)

In [61]:
# 시도 문제 중 가장 낮은 레벨 추출
tried_levels = {}

for tried_tag in tried_list:
    cond1 = (dum['status']=='tried')
    cond2 = (dum['bojTagId']==tried_tag)
    cond_list = list(dum.loc[cond1&cond2]['level'])
    tried_levels[tried_tag] = int(sum(cond_list)/len(cond_list))
#     tried_levels[tried_tag] = sorted(list(dum.loc[cond1&cond2]['level']))[-1]

In [62]:
print('푼 문제 태그 목록 : ', solved_list)
print('푼 문제 난이도(내림차순) : ', solved_levels)
print()
print('시도 문제 태그 목록 : ', tried_list)
print('시도 문제 난이도(오름차순) : ', tried_levels)

푼 문제 태그 목록 :  [124, 102, 175, 121, 97, 125, 7, 158, 11, 126, 71, 127, 72, 5, 12, 25, 141, 95, 33, 62, 170, 136, 9, 67, 100, 6, 120, 24, 14, 117, 26, 8, 60, 73]
푼 문제 난이도(내림차순) :  {124: 3, 102: 5, 175: 7, 121: 2, 97: 6, 125: 7, 7: 9, 158: 5, 11: 10, 126: 10, 71: 7, 127: 9, 72: 7, 5: 9, 12: 7, 25: 6, 141: 10, 95: 7, 33: 7, 62: 9, 170: 8, 136: 7, 9: 7, 67: 7, 100: 3, 6: 5, 120: 8, 24: 9, 14: 6, 117: 7, 26: 6, 8: 4, 60: 3, 73: 7}

시도 문제 태그 목록 :  [121, 124, 136, 175]
시도 문제 난이도(오름차순) :  {121: 2, 124: 2, 136: 7, 175: 7}


In [63]:
tag_list = tried_list[:5] + solved_list[:5]
tag_list = list(set(tag_list))

In [64]:
tag_list

[97, 102, 136, 175, 121, 124]

In [123]:
recomm_df = pd.DataFrame(columns=df.columns)

In [124]:
# 푼 문제 제외
df = df.loc[~df['problemId'].isin(solved_problems)]

In [125]:
for tag in tag_list:
    if tag in solved_levels:
        tag_level = solved_levels[tag]
    elif tag in tried_levels:
        tag_level = tried_levels[tag]
    else:
        print('error')
        tag_level = 0        
    print('tag: ', tag, 'tag level: ', tag_level)
    
    recomm_tag_level = [tag_level]
    for i in range(2):
        recomm_tag_level.append(tag_level+(i+1))
        if tag_level-(i+1) >= 0:
            recomm_tag_level.append(tag_level-(i+1))
    print('recomm tag levels: ', sorted(recomm_tag_level))
    
    tmp_df = df.loc[(df['bojTagId']==tag)&(df['level'].isin(recomm_tag_level))]
    tmp_df = tmp_df.sort_values(['acceptedUserCount', 'averageTries'], ascending=[False, True])[:10]
    
#     recomm_df = recomm_df.append(tmp_df)
    recomm_df = pd.concat([recomm_df, tmp_df])

tag:  97 tag level:  6
recomm tag levels:  [4, 5, 6, 7, 8]
tag:  102 tag level:  5
recomm tag levels:  [3, 4, 5, 6, 7]
tag:  136 tag level:  7
recomm tag levels:  [5, 6, 7, 8, 9]
tag:  175 tag level:  7
recomm tag levels:  [5, 6, 7, 8, 9]
tag:  121 tag level:  2
recomm tag levels:  [0, 1, 2, 3, 4]
tag:  124 tag level:  3
recomm tag levels:  [1, 2, 3, 4, 5]


In [127]:
recomm_df = recomm_df.drop_duplicates('problemId').reset_index()

In [128]:
recomm_df = recomm_df.sort_values(['acceptedUserCount', 'averageTries'], ascending=[False, True])

In [129]:
recomm_json = recomm_df.to_json(orient='records')

In [130]:
print(json.dumps(json.loads(recomm_json), indent="\t") )

[
	{
		"index": 3,
		"problemId": 1001,
		"titleKo": "A-B",
		"titles": "[{'language': 'ko', 'languageDisplayName': 'ko', 'title': 'A-B', 'isOriginal': True}]",
		"isSolvable": true,
		"isPartial": false,
		"acceptedUserCount": 146249,
		"level": 1,
		"votedUserCount": 8,
		"sprout": false,
		"givesNoRating": false,
		"isLevelLocked": true,
		"averageTries": 1.3962,
		"official": true,
		"tags": "[{'key': 'arithmetic', 'isMeta': False, 'bojTagId': 121, 'problemCount': 585, 'displayNames': [{'language': 'en', 'name': 'arithmetic', 'short': 'arithmetic'}, {'language': 'ko', 'name': '\uc0ac\uce59\uc5f0\uc0b0', 'short': '\uc0ac\uce59\uc5f0\uc0b0'}], 'problemId': 1001, 'tag_name': '\uc0ac\uce59\uc5f0\uc0b0', 'tag_short': '\uc0ac\uce59\uc5f0\uc0b0'}, {'key': 'implementation', 'isMeta': False, 'bojTagId': 102, 'problemCount': 3456, 'displayNames': [{'language': 'en', 'name': 'implementation', 'short': 'impl'}, {'language': 'ko', 'name': '\uad6c\ud604', 'short': '\uad6c\ud604'}], 'problemId': 